# Génération de réponses originales

Modèle

In [1]:
import pandas as pd

data = pd.read_csv('C:/Users/mchampain/Documents/GitHub/de-fr.txt/echantillon.txt', header = None, sep='\t')

Setup - Import TensorFlow and other libraries

In [2]:
#conda install tensorflow

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time

In [4]:
text = open('C:/Users/mchampain/Documents/GitHub/de-fr.txt/echantillon.txt', 'rb').read().decode(encoding='utf-8')

In [5]:
print ('Length of text: {} characters'.format(len(text)))

Length of text: 4468751 characters


In [6]:
print(text[:250])

Intéressant, ton choix de mots. Putain, elle est la seule chose qui les intéresse. Remets ton cache-nez ! Il faudra lui parler, Mme Kane. us fort, blanc-bec. C'était un très beau match. Vous avez raison. Tu dois apprendre comment... Mon scrotum  n'es


In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))


205 unique characters


Process the text
Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [8]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [9]:
import pickle

with open('char2idx.pkl','wb') as f:
    pickle.dump(char2idx, f)

with open('idx2char.pkl','wb') as f:
    pickle.dump(idx2char, f)

In [10]:
text_as_int

array([37, 74, 80, ..., 75, 74, 13])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to len(unique).

In [11]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')


{
  ' ' :   0,
  '!' :   1,
  '"' :   2,
  '#' :   3,
  '$' :   4,
  '%' :   5,
  "'" :   6,
  '(' :   7,
  ')' :   8,
  '*' :   9,
  '+' :  10,
  ',' :  11,
  '-' :  12,
  '.' :  13,
  '/' :  14,
  '0' :  15,
  '1' :  16,
  '2' :  17,
  '3' :  18,
  '4' :  19,
  ...
}


In [12]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))


'Intéressant, ' ---- characters mapped to int ---- > [ 37  74  80 144  78  65  79  79  61  74  80  11   0]


The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?
Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [16]:
char2idx

{' ': 0,
 '!': 1,
 '"': 2,
 '#': 3,
 '$': 4,
 '%': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 '*': 9,
 '+': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '=': 27,
 '@': 28,
 'A': 29,
 'B': 30,
 'C': 31,
 'D': 32,
 'E': 33,
 'F': 34,
 'G': 35,
 'H': 36,
 'I': 37,
 'J': 38,
 'K': 39,
 'L': 40,
 'M': 41,
 'N': 42,
 'O': 43,
 'P': 44,
 'Q': 45,
 'R': 46,
 'S': 47,
 'T': 48,
 'U': 49,
 'V': 50,
 'W': 51,
 'X': 52,
 'Y': 53,
 'Z': 54,
 '[': 55,
 '\\': 56,
 ']': 57,
 '^': 58,
 '_': 59,
 '`': 60,
 'a': 61,
 'b': 62,
 'c': 63,
 'd': 64,
 'e': 65,
 'f': 66,
 'g': 67,
 'h': 68,
 'i': 69,
 'j': 70,
 'k': 71,
 'l': 72,
 'm': 73,
 'n': 74,
 'o': 75,
 'p': 76,
 'q': 77,
 'r': 78,
 's': 79,
 't': 80,
 'u': 81,
 'v': 82,
 'w': 83,
 'x': 84,
 'y': 85,
 'z': 86,
 '{': 87,
 '}': 88,
 '~': 89,
 '\x81': 90,
 '\x87': 91,
 '\x88': 92,
 '\x8d': 93,
 '\x8e': 94,
 '\x8f': 95,
 '\x90': 96,
 '\x93': 97,
 '\x9

In [13]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])


I
n
t
é
r


The batch method lets us easily convert these individual characters to sequences of the desired size.

In [14]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))


'Intéressant, ton choix de mots. Putain, elle est la seule chose qui les intéresse. Remets ton cache-n'
"ez ! Il faudra lui parler, Mme Kane. us fort, blanc-bec. C'était un très beau match. Vous avez raison"
". Tu dois apprendre comment... Mon scrotum  n'est pas très sur. Alors, vous étiez amis toi et le garg"
"arisme. Abandonnez. Inutile, me voici. Bon, peut-être que je n'aurais pas dû en parler maintenant. Ce"
" n'était pas mon intention. n, je ne parle pas ukrainien, mais je parle enveloppe de cash. Je parie q"


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [15]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [16]:
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int32, tf.int32)>

Print the first examples input and target values:

In [17]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))


Input data:  'Intéressant, ton choix de mots. Putain, elle est la seule chose qui les intéresse. Remets ton cache-'
Target data: 'ntéressant, ton choix de mots. Putain, elle est la seule chose qui les intéresse. Remets ton cache-n'


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the RNN considers the previous step context in addition to the current input character.

In [18]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))


Step    0
  input: 37 ('I')
  expected output: 74 ('n')
Step    1
  input: 74 ('n')
  expected output: 80 ('t')
Step    2
  input: 80 ('t')
  expected output: 144 ('é')
Step    3
  input: 144 ('é')
  expected output: 78 ('r')
Step    4
  input: 78 ('r')
  expected output: 65 ('e')


Create training batches

We used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [19]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

Build The Model

Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

    tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;
    tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use a LSTM layer here.)
    tf.keras.layers.Dense: The output layer, with vocab_size outputs.


In [20]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024


In [21]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [22]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)


For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

A drawing of the data passing through the model
Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [23]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 205) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is 100 but the model can be run on inputs of any length:

In [24]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           52480     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 205)           210125    
Total params: 4,200,909
Trainable params: 4,200,909
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.
Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [25]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()


This gives us, at each timestep, a prediction of the next character index:

In [26]:
sampled_indices

array([109, 183, 201,  13,  91, 159,  79, 153, 168,  58,  53, 172, 141,
        75,  42,   8, 180,  97, 113, 115, 123, 107, 138,   1,  17, 135,
       114, 165,  78,  42,  84, 101, 119,  56, 119,  17, 167,  22,  75,
        52, 165, 197,  57,  38,  87,  91, 192, 186, 114, 184,  75, 150,
       157,   0, 182,  68,  80,  75,  91,  33,  19, 155,  69,  17, 110,
       153, 167, 128,  84, 113,  25,  66, 117, 115, 145, 163,  82, 180,
       142,   7, 154, 169,  10, 175, 194,  84,  71, 118, 136, 115, 129,
        65,  77, 191, 115,  40,  46,  78, 100, 162], dtype=int64)

Decode these to see the text predicted by this untrained model:

In [27]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))


Input: 
 "ça me fait...  tapis ! Les Chinois continuent leur enquête sur notre implication dans l'attaque de l"

Next Char Predictions: 
 '©Ѕ♪.\x87üsôű^YΑåoN)ο\x93°´Á§â!2ß²œrNx\x9dº\\º2ů7oXœ‡]J{\x87\u200bи²аoïú ϋhto\x87E4øi2ªôůÈx°:f¸´êıvοç(öž+δ–xk¹à´Éeqі´LRr\x9cę'


Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.
Attach an optimizer, and a loss function

The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the from_logits flag.

In [28]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())


Prediction shape:  (64, 100, 205)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.32418


Configure the training procedure using the tf.keras.Model.compile method. We'll use tf.keras.optimizers.Adam with default arguments and the loss function.

In [29]:
model.compile(optimizer='adam', loss=loss)


Configure checkpoints
Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [30]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [31]:
EPOCHS=10

In [32]:
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [33]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
691/691 [==============================] - 3490s 5s/step - loss: 1.9918
Epoch 2/10
691/691 [==============================] - 3480s 5s/step - loss: 1.4066
Epoch 3/10
691/691 [==============================] - 3481s 5s/step - loss: 1.3016
Epoch 4/10
691/691 [==============================] - 3214s 5s/step - loss: 1.2511
Epoch 5/10
691/691 [==============================] - 2285s 3s/step - loss: 1.2180
Epoch 6/10
691/691 [==============================] - 2463s 4s/step - loss: 1.1923
Epoch 7/10
691/691 [==============================] - 2403s 3s/step - loss: 1.1714
Epoch 8/10
691/691 [==============================] - 2430s 4s/step - loss: 1.1540
Epoch 9/10
691/691 [==============================] - 2406s 3s/step - loss: 1.1388
Epoch 10/10
691/691 [==============================] - 2060s 3s/step - loss: 1.1257


In [35]:
model_json = model.to_json()
with open("model_generation_text.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_generation_text.h5")

Generate text
Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [36]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_10'

In [37]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))


In [38]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            52480     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 205)            210125    
Total params: 4,200,909
Trainable params: 4,200,909
Non-trainable params: 0
_________________________________________________________________


The prediction loop

The following code block generates the text:

    It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

    Get the prediction distribution of the next character using the start string and the RNN state.

    Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

    The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.

To generate text the model's output is fed back to the input

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [39]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 100

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))


In [42]:
print(generate_text(model, start_string=u"Bonjour"))


Bonjour moi, oui... Primous depuis le début ! On pourrait aller vivant ce qu'il pense. À mon réveillet, tou
